Here is the notebook for using Dedupe library in the Microsoft Fabric.  
The main benefits of this notebook is to avoid problem with interactivity of dedupe.console_label(deduper)  
Dedupe library has active learning algorithm, so it is necessary to mark pairs after each input. 

## Import libraries

In [ ]:
## Import libraries
!pip install dedupe

In [ ]:
%pip install ipywidgets

In [ ]:
# Now launch widget UI labeling
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
import dedupe
import pandas as pd
import numpy as np
from dedupe import variables

## Load data

In [ ]:
df = spark.sql("SELECT * FROM Site_Project.final_union_personnel_clean  LIMIT ALL").toPandas()

In [ ]:
cols_to_clean = [
    "address_street_1_clean",
    "address_street_2_clean",
    "city_clean",
    "state_clean",
    "country_clean"
]

df[cols_to_clean] = df[cols_to_clean].replace(r'^\s*$', pd.NA, regex=True)

# Step 2: Combine address parts (without empty strings), using string join logic
df["full_address_clean"] = df[cols_to_clean].apply(
    lambda row: ", ".join(row.dropna().astype(str)), axis=1
)

df["full_address_clean"] = df["full_address_clean"].replace(r'^\s*$', pd.NA, regex=True)

In [ ]:
(df.astype(str).apply(lambda x: x.str.strip() == "")).sum()

In [ ]:
# Step 1: Create a unique ID column for dedupe
df['record_id'] = df.index.astype(str)  # create from row index

In [ ]:
#Null and Distinct counts function
def summarize_nulls(df, df_name):
    total_rows = len(df)
    null_count = df.isnull().sum()
    distinct_count = df.nunique()
    null_counts_summary = pd.DataFrame( {
        "Missing Values" : null_count,
        "Percentage": (null_count/total_rows * 100).round(2),
        "Distinct Values" : distinct_count,
        "Percentage Distinct of Non Missing" : (distinct_count/(total_rows-null_count) * 100).round(2),
        "Percentage Distinct" : (distinct_count/total_rows * 100).round(2),
        "Total Rows" : total_rows
    }).sort_values(by='Missing Values', ascending=False)
    print(f"Null counts in '{df_name}' by column (highest to lowest):")
    print(null_counts_summary)

In [ ]:
pd.set_option('display.max_columns', None)         # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to the next line
pd.set_option('display.max_colwidth', None)        # Show full content of each cell
summarize_nulls(df, "union_personel")

## Dedupe data

In [ ]:
#STEP 2: Fields used in dedupe ✅
dedupe_fields = [
    'full_name_clean',
    'name_prefix_clean',
    'name_suffix_clean',
    'email_clean', 
    'phone_clean',
    'fax_number_clean',
    'address_street_1_clean',
    'city_clean',
    'state_clean',
    'country_clean',
    'postal_code_clean',
    'full_address_clean'
]

In [ ]:
# change NA to None, requirement from dedupe library
df = df.applymap(lambda x: None if pd.isna(x) else x)

In [ ]:
# Step 2: Convert fields to dict-of-dicts format as required in dedupe documentation
data_d = df.set_index('record_id')[dedupe_fields].to_dict(orient='index')

In [ ]:
# for inspection 
for k in list(data_d)[:10]:
    print(f"{k}: {data_d[k]}")

## Dedupe

In [ ]:
# Defining type of each variable for dedupe ✅
variable_definition = [
    variables.String("full_name_clean", has_missing=True),
    variables.ShortString("name_prefix_clean", has_missing=True),
    variables.ShortString("name_suffix_clean", has_missing=True),
    variables.String("email_clean", has_missing=True),
    variables.String("phone_clean", has_missing=True),
    variables.String("fax_number_clean", has_missing=True),
    variables.String("address_street_1_clean", has_missing=True),
    variables.ShortString("city_clean", name="city", has_missing=True),
    variables.ShortString("state_clean", has_missing=True),
    variables.Exact("country_clean", has_missing=True),
    variables.ShortString("postal_code_clean", has_missing=True),
    variables.Text("full_address_clean", has_missing=True)
]

In [ ]:
#Initialize deduper with variable definition
deduper = dedupe.Dedupe(variable_definition)

In [ ]:
#Start active learning for labeling pairs (interactive labeling will start, but here faked it)
deduper.prepare_training(data_d, sample_size=10000)

In [ ]:
labeled = {'match': [], 'distinct': [], 'unsure': []}
index = 0
stop_labeling = False  # Global control flag

In [ ]:
def show_pair(i):
    if stop_labeling:
        print("✅ Labeling stopped. You can now train your model.")
        return

    pairs = deduper.uncertain_pairs()

    rec1, rec2 = pairs[0]

    clear_output()
    print("\n🟦 Record 1:")
    for k, v in rec1.items(): print(f"  {k}: {v}")
    print("\n🟥 Record 2:")
    for k, v in rec2.items(): print(f"  {k}: {v}")
    print("\nIs this a match?")

    btn_yes = widgets.Button(description="✅ Yes")
    btn_no = widgets.Button(description="❌ No")
    btn_unsure = widgets.Button(description="❓ Unsure")
    btn_exit = widgets.Button(description="⏹ Exit Labeling")

    def on_click_factory(label):
        def on_click(b):
            labeled[label].append((rec1, rec2))
            next_pair()
        return on_click
    
    def on_exit(b):
        global stop_labeling
        stop_labeling = True
        print("✅ Labeling stopped. You can now train your model.")

    btn_yes.on_click(on_click_factory("match"))
    btn_no.on_click(on_click_factory("distinct"))
    btn_unsure.on_click(on_click_factory("unsure"))
    btn_exit.on_click(on_exit)

    display(widgets.HBox([btn_yes, btn_no, btn_unsure, btn_exit]))

def next_pair():
    global index
    index += 1
    #Giving labeled results for training
    deduper.mark_pairs(labeled)
    show_pair(index)

# Start labeling
show_pair(index)

In [ ]:
# Step 6: Train after labeling
deduper.train()

## Clustering

In [ ]:
# Step 7: Match and cluster
# threshold = deduper.best_threshold()
clusters = deduper.partition(data_d, threshold = 0.5)

In [ ]:
# Step 9: Map cluster IDs back to the DataFrame
cluster_membership = {}
for cluster_id, (record_ids, confidence) in enumerate(clusters):
    for record_id in record_ids:
        cluster_membership[record_id] = cluster_id

df['cluster_id'] = df['record_id'].map(cluster_membership)

In [ ]:
df.shape

## Save results

In [ ]:
#Save resulting df into a table in Lakehouse
spark_df = spark.createDataFrame(df)
# Save as a Delta table in your Lakehouse workspace
spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dedupe_data_people")

In [ ]:
df_check = spark.sql("SELECT * FROM Site_Project.dedupe_data_people  LIMIT ALL").toPandas()
df_check.shape